# Visualizing H3 Cells with Data Explorer and Python

This notebook is designed to show you what is involved in querying geospatial data from Azure Data Explorer and rendering the results using different Python libraries and Jupyter extensions.

For instructions on setting up your environment to run these demos, please see: https://github.com/drewfurgiuele/adx-h3-viz/blob/main/README.md

## Importing Required Libraries

There are two demos in this notebook; one uses Plotly and the other uses Kepler.gl. The following cell imports libraries to support both demos.

In [ ]:
import pandas
import numpy
import json
import geojson
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from keplergl import KeplerGl

## Setting up varibles for Azure Data Explorer connection

The following cells sets your cluster and database settings for connecting via the Python SDK for Kusto. Note that you will need to provide your cluster URL below, replacing the "<your cluster here>" section with your specific URL.

In [ ]:
AAD_TENANT_ID = "common"
KUSTO_CLUSTER = "https://kvcd43df9301b3242b991a.southcentralus.kusto.windows.net/"
KUSTO_DATABASE = "opencellid"

In [ ]:
KCSB = KustoConnectionStringBuilder.with_aad_device_authentication(KUSTO_CLUSTER)
KCSB.authority_id = AAD_TENANT_ID
KUSTO_CLIENT = KustoClient(KCSB)

## Executing your Kusto query

This cell will query your database. Note that in the query below, substitute your MCC query to whatever dataset you loaded. The example below is for The Netherlands (MCC==204) so change it according to what you want to see. Notice the last ```extend``` statement: We're adding the required structure used by the GeoJSON standard by using the ```pack()``` function.

When you run the cell, a new web browser will open asking you to authenticate yourself. You will need the code at the bottom of the cell, and then you'll follow the normal authentication flow.

In [ ]:
KUSTO_QUERY = """OpenCellH3Data
| where MCC == 204
| where isnotempty(H3_Medium)
| summarize arg_max(Updated, *) by RadioType, MCC, MNC, LAC
| summarize SignalStrength=avg(AvgSignal), dcount(LAC), UmtsCells = countif(RadioType == "UMTS"), LTE = countif(RadioType == "LTE") by H3_Medium
| extend H3_Hash_Polygon = geo_h3cell_to_polygon(H3_Medium)
| extend Features=pack(
    "geometry", H3_Hash_Polygon,
    "id", H3_Medium,
    "properties",pack("value", UmtsCells),
    "type", "Feature")"""

RESPONSE = KUSTO_CLIENT.execute(KUSTO_DATABASE, KUSTO_QUERY)

## Creating the GeoJSON FeatureList

To properly format the GeoJSON object for rendering the results, the following two cells will take the Kusto result set and convert it to a GeoJSON feature list. This object will then be used later on in the charts.

In [ ]:
df = dataframe_from_result_table(RESPONSE.primary_results[0])
feature_list = df['Features'].to_list()

In [ ]:
from geojson import FeatureCollection
feat_collection = FeatureCollection(feature_list)

## Plotting H3 with Plotly

Plotly provides the choropleth_mapbox method for rendering the data. At a high level, we use the feature collection we created above to set the geojson value, and set the "colors" of the cells to measure the different ranges of the indexes. The one thing to note here is that this map does not auto-center on your data; therefore, if you use a different MCC code you'll need to set the center values to set the map location. Finally, we're setting the "hover_data" array to tell the map which properties to show when someone places their cursor over the cells.

In [ ]:
fig = (px.choropleth_mapbox(
    df, 
    geojson=feat_collection, 
    locations='H3_Medium', 
    color='UmtsCells',
    color_continuous_scale="agsunset",
    range_color=(0,df.UmtsCells.mean()),                  
    mapbox_style='carto-positron',
    zoom=6.5,
    center = {"lat": 52.1326, "lon": 5.2913},            
    opacity=0.6,
    hover_data=['SignalStrength','dcount_LAC','UmtsCells','LTE'],
    width=700,
    height=800,
))

In [ ]:
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show() 

## Plotting H3 Data with Kepler.gl

Plotting the same results set with Kepler.gl works much the same way. First, initalize the map with your desired height and width. Next, call the ```add_data()``` method to add the feature collection to the map, and to name the dataset. Once the data is loaded, you can customer the map colors, rotation, height of the cells, etc from the built-in control panel.

In [ ]:
map_2 = KeplerGl(height=600)
map_2.add_data(data=json.dumps(feat_collection), name='NL Cell Coverage')
map_2

### Exporting and using your map configuration

Once you have your plotted data looking the way you'd like, you can use the .config property to get the saved styles and map settings. You can then load that configuration when you create another map

In [ ]:
map_config = map_2.config

In [ ]:
map_3 = KeplerGl(height=600)
map_3.add_data(data=json.dumps(feat_collection), name='NL Cell Coverage')
map_3.config = map_config
map_3